In [1]:
import os
import base64
import pandas as pd

from textual_features.agent_extraction import Llm, BP_Metrics_Getter

[nltk_data] Downloading package punkt_tab to
[nltk_data]     c:\Users\LENOVO\anaconda3\envs\bp_digitalization\Lib\s
[nltk_data]     ite-packages\llama_index\core\_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
from textual_features.prompts_and_parsers import Llm_Extraction_Prompt

In [3]:
from azure_authentication.customized_azure_login import CredentialFactory

In [4]:
import nest_asyncio

In [ ]:
def create_llm_costs_dict(llm):
    """Creates a dictionary with the costs of the LLM."""
    llm_costs = {
        "llm_prompt_tokens": llm.token_counter.prompt_llm_token_count,
        "llm_completion_tokens": llm.token_counter.completion_llm_token_count,
        "total_llm_token_count": llm.token_counter.total_llm_token_count,
        "total_llm_costs_in_euro": llm.calculate_llm_calling_price(llm.token_counter.prompt_llm_token_count,
                                                                   llm.token_counter.completion_llm_token_count),
    }

    return llm_costs

In [5]:
os.chdir("../")
CWD = os.getcwd()

data_dir = os.path.join(CWD, 'data')

#Specify mode (working with a sample or all the files?)
sample_mode = False 
sample_size = 50

# specify file path
INPUT_FILE_PATH = os.path.join(CWD, "data", "proc", "building_plans_sample", "test_images", "bp_text.json")
METADATA_PATH = os.path.join(CWD, "data", "proc", "building_plans", "metadata","building_plans_metadata.csv")

# specify relevant column names
ID_COLUMN='filename'
TEXT_COLUMN='content'

# read in data
bp_text = pd.read_json(INPUT_FILE_PATH)
metadata_df = pd.read_csv(METADATA_PATH)

In [6]:
metadata_bps = metadata_df[metadata_df['Planart'].isin(['qualifizierter Bebauungsplan', 'einfacher Bebauungsplan', 'vorhabenbezogener Bebauungsplan'])]

In [7]:
bp_text['id'] = bp_text['filename'].str.extract(r'(\d+)_').astype(int)

In [8]:
#input_df = metadata_bps.merge(bp_text)

In [9]:
input_df = bp_text

In [10]:
# Recommendation: Configure your own authentication workflow with environment variables, see the description at
# https://github.com/soda-lmu/azure-auth-helper-python/blob/main/AuthenticationWorkflowSetup.md
credential = CredentialFactory().select_credential()
token_provider = credential.get_login_token_to_azure_cognitive_services()

print("Instantiate Azure OpenAI Client")

Instantiate Azure OpenAI Client


In [11]:
if sample_mode:
    
    run_data = input_df.sample(sample_size, random_state=15)

else: 
 
    run_data = input_df

In [12]:
llm = Llm(model_name="gpt-4-1106-preview")
getter = BP_Metrics_Getter(llm = llm, llm_single_prompt = Llm_Extraction_Prompt())

In [16]:
run_data

,filename,content,id
0,695_0.jpg,"[Bebauungsplan (BBP) ""Sonnenschein"", PRÄAMBEL,...",695
1,807_0.jpg,"[Zeichenerklärung für Festsetzungen, Hinweise,...",807
2,904_0.jpg,"[Gemeinde Stadtlauringen, OT Altenmünster, Fes...",904
3,904_1.jpg,"[5.0, Höhenfestsetzungen (8 9, Abs., 2 BauGB),...",904
4,904_2.jpg,"[8.0, Textliche Festsetzung, der, Grünordnun...",904
5,904_3.jpg,"[8.4, Vollzugsfrist, verbindlichen Anpflanzun...",904
6,904_4.jpg,[ZEICHENERKLÄRUNG ZU DEN PLANLICHEN FESTSETZU...,904
7,904_5.jpg,"[Verkehrsflächen, Abs, BauGB), Öffentliche St...",904
8,904_6.jpg,"[13.2 Sonstige Grünordnung, Ads,, 250, und 2...",904
9,904_7.jpg,"[INDERUNGSVERMERK, GEA, NANE, GeFR, Nave, Froj...",904


In [13]:
nest_asyncio.apply()

results = pd.DataFrame(columns=["id", "output", "llm_prompt_tokens", "llm_completion_tokens", "total_llm_token_count", "total_llm_costs_in_euro"])

for i, row in run_data.iterrows():
    
    response = await getter._bound_get_emissions_from_raw_text(row['content'])
    completion_tokens = response.usage.completion_tokens

    price = calculate_llm_calling_price(deployment_name, prompt_tokens, completion_tokens)
    
    parsed_extractions = getter._parse_to_classes_llm_output(response)
    
    #results.append(parsed_extractions)

    row = pd.DataFrame({"filename": row['filename'], "output": parsed_extractions, 
                         "llm_prompt_tokens": llm_costs["llm_prompt_tokens"], "llm_completion_tokens" : llm_costs["llm_completion_tokens"], 
                         "total_llm_token_count": llm_costs["total_llm_token_count"], "total_llm_costs_in_euro" : llm_costs["total_llm_costs_in_euro"]})

    results = pd.concat([results, row])

Current Time: 12:32:09 ( 0.00017503333316805462 minutes since process started.)
LLM query execution time: 5.212466199998744 seconds (5.222969900001772 seconds since initiating ValueRetrieverPipeline).
Current Time: 12:32:14 ( 0.08711057999947419 minutes since process started.)
LLM query execution time: 6.460410400060937 seconds (11.68704600003548 seconds since initiating ValueRetrieverPipeline).
Current Time: 12:32:20 ( 0.19480677166720853 minutes since process started.)
LLM query execution time: 5.067159299971536 seconds (16.755566300009377 seconds since initiating ValueRetrieverPipeline).
Current Time: 12:32:25 ( 0.2792758100006419 minutes since process started.)
LLM query execution time: 3.909944299957715 seconds (20.66649370000232 seconds since initiating ValueRetrieverPipeline).
Current Time: 12:32:29 ( 0.34445869000046514 minutes since process started.)
LLM query execution time: 5.669114999938756 seconds (26.336636999971233 seconds since initiating ValueRetrieverPipeline).
Curren

In [14]:
results

[BuildingMetrics(grz=GRZ(value=0.35), gfz=GFZ(value=0.35)),
 BuildingMetrics(grz=GRZ(value=0.6), gfz=GFZ(value=1.0)),
 BuildingMetrics(grz=GRZ(value=0.6), gfz=GFZ(value=2.0)),
 BuildingMetrics(grz=GRZ(value=None), gfz=GFZ(value=None)),
 BuildingMetrics(grz=GRZ(value=None), gfz=GFZ(value=None)),
 BuildingMetrics(grz=GRZ(value=None), gfz=GFZ(value=None)),
 BuildingMetrics(grz=GRZ(value=0.6), gfz=GFZ(value=2.0)),
 BuildingMetrics(grz=GRZ(value=None), gfz=GFZ(value=None)),
 BuildingMetrics(grz=GRZ(value=None), gfz=GFZ(value=None)),
 BuildingMetrics(grz=GRZ(value=None), gfz=GFZ(value=None)),
 BuildingMetrics(grz=GRZ(value=None), gfz=GFZ(value=None))]

In [16]:
parsed_extractions

BuildingMetrics(grz=GRZ(value=0.35), gfz=GFZ(value=0.35))

In [17]:
#OUTPUT_FLOODING_FILE_PATH = os.path.join("data", "proc", "building_plans_sample", "features",  "test_images_flooding_data_extraction.csv")
#OUTPUT_EXTRACTIONS_FILE_PATH = os.path.join("data", "proc", "building_plans_sample", "features",  "test_images_info_data_extraction.csv")
#
#
#extraction_df.to_csv(OUTPUT_EXTRACTIONS_FILE_PATH)
#flooding_df.to_csv(OUTPUT_FLOODING_FILE_PATH)